In [6]:
import os
import laspy
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm import tqdm

In [7]:
STEP = 20

In [8]:
eval_folder = Path('./eval')
eval_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
input_folder = eval_folder / 'clouds_to_process'
output_folder = eval_folder / 'clouds_processed'
output_folder.mkdir(parents=True, exist_ok=True)

extensions = ('.laz', '.las')
point_clouds = sorted(
            [f for f in input_folder.rglob("*") if f.is_file() and f.suffix.lower() in extensions],
            key=lambda f: f.name
        )

for i, path in enumerate(point_clouds):
    ext = path.suffix.lower()
    file = laspy.read(path)

    points = file.points
    coords = np.vstack((file.x, file.y, file.z)).transpose()
    intensity = np.array(file.intensity)

    idx = np.floor_divide(coords, STEP).astype(int)
    idx -= idx.min(axis=0)
    dims = idx.max(axis=0) + 1

    idx = np.ravel_multi_index(idx.T, dims)
    unique_idx, inverse_map = np.unique(idx, return_inverse=True)

    for i, chunk_idx in enumerate(tqdm(unique_idx)):
        mask = idx == chunk_idx
        chunk_intensities = intensity[mask]
        file.points = points[mask]
    
        min_intensity = np.min(chunk_intensities)
        max_intensity = np.max(chunk_intensities)
        file.add_extra_dim(laspy.ExtraBytesParams(name="I_norm", type=np.float32))
        file.I_norm = (chunk_intensities - min_intensity) / (max_intensity - min_intensity)
        file.write(output_folder / f'plot_{i}{ext}')


KeyboardInterrupt: 